In [1]:
# Preparation
from acquire_msc import get_all_readme_files_and_languages
# from acquire_readme_files import temp_get_all_readme_files_and_languages
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd

In [2]:
url_list = ['microsoft/Terminal', 'jackfrued/Python-100-Days', 
            'kkuchta/css-only-chat', 'microsoft/PowerToys', 'jolaleye/cssfx', 
            'MisterBooo/LeetCodeAnimation', 'flutter/flutter_web', 
            'TheAlgorithms/Python', 'hiroppy/fusuma', 'CyC2018/CS-Notes', 
            'jaywcjlove/linux-command', 'flutter/flutter', '996icu/996.ICU', 
            'STVIR/pysot', 'minamarkham/formation', 'azl397985856/leetcode', 
            'qianguyihao/Web', 'react-native-windows', 'sql-machine-learning/sqlflow', 
            'sabakkps/backslide', 'dgryski/go-perfbook', 'Snailclimb/JavaGuide', 
            'microsoft/vscode', 'markphelps/flipt', 'teoga/awesome-product-design']

In [3]:
def basic_clean(article):
    '''
    take in a string (article) and return it after applying some basic text cleaning to it:
        - lowercase everything
        - normalize unicode characters
        - replace anything that is not a letter, number, whitespace or a single quote
    '''
    new_article = article.lower()
    new_article = re.sub(r'\s', ' ', new_article)
    normalized = unicodedata.normalize('NFKD', new_article)                .encode('ascii', 'ignore')                .decode('utf-8')
    without_special_chars = re.sub(r'[^\w\s]', ' ', normalized)
    word_list = without_special_chars.split()
    word_list = ' '.join(word_list)
    return word_list

In [4]:
def tokenize(article):
    '''tokenize all the words in the string, article'''
    tokenizer = nltk.tokenize.ToktokTokenizer()
    new_article = tokenizer.tokenize(article, return_str=True)
    return new_article

In [5]:
def print_stop_words(article):
    '''accept some text, apply stemming to all of the words,
        and print a list of value counts for all the stemmed words'''
    # Create the nltk stemmer object, then use it
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in article.split()]
    print(pd.Series(stems).value_counts())

In [6]:
def stem(article):
    '''accept a string and return it after applying stemming to all the words'''
    ps = nltk.stem.PorterStemmer()
    article_stemmed = ''.join([ps.stem(word) for word in article])
    return article_stemmed

In [7]:
def lemmatize(article):
    '''accept a string and return it after applying lemmatization to each word.'''
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized_words = [wnl.lemmatize(word) for word in article]
    article_lemmatized = ''.join(lemmatized_words)
    return article_lemmatized

In [8]:
def remove_stopwords(article, extra_words = None, exclude_words = None):
    '''remove all the stopwords, including all the words in extra_words and excluding
    all the words in exclude list'''

    # get basic stopword list
    stopword_list = stopwords.words('english')

    # add extra words    
    if extra_words != None:
        stopword_list = stopword_list + extra_words
    # remove excluded words
    if exclude_words != None:
        stopword_list = [word for word in stopword_list if word not in exclude_words]
    
    without_stopwords = [word for word in article.split(' ') if word not in stopword_list]
    article_without_stopwords = ' '.join(without_stopwords)
    return article_without_stopwords

In [9]:
def prep_article(this_dict, extra_words = None, exclude_words = None):
    '''
    takes in a dictionary representing an article and returns a dictionary that 
    looks like this:
            {
             'title': 'the original title',
             'original': original,
             'stemmed': article_stemmed,
             'lemmatized': article_lemmatized,
             'clean': article_without_stopwords
            }
    Note that if the orignal dictionary has a title property, it will remain unchanged 
    (same goes for the category property).
    '''
    # put the content section into article and make a copy
#     article = articles['content'][article_index]    # needed for 
    article = this_dict['content']
    original = article

    '''
    apply some basic text cleaning to the string, article:
        - lowercase everything
        - normalize unicode characters
        - replace anything that is not a letter, number, whitespace or a single quote
    '''
    article = basic_clean(article)

    '''tokenize all the words in the string, article'''
    article = tokenize(article)

    '''applying stemming to all the words in the string, article'''
    article_stemmed = stem(article)
    
    ''''apply lemmatization to each word in the string, article'''
    article_lemmatized = lemmatize(article)
    
    '''remove all the stopwords, including all the words in extra_words and excluding
    all the words in exclude list'''
    article_without_stopwords = remove_stopwords(article, extra_words, exclude_words)

    keys = list(this_dict.keys())
    
    new_dict = {
         'title': this_dict['title'],
         'original': original,
         'category': [this_dict['category'] if 'category' in keys else 'repo_readme'],
         'stemmed': article_stemmed,
         'lemmatized': article_lemmatized,
         'clean': article_without_stopwords
        }
    return new_dict

In [10]:
def prepare_article_data(articles, extra_words = None, exclude_words = None):
    # takes in the list of articles dictionaries, 
    # applies the prep_article function to each one, 
    # and returns the transformed data.
    transformed_articles = []

    for article_index in range(len(articles)):
        transformed_entry = prep_article(articles[article_index], extra_words, exclude_words)
        transformed_articles.append(transformed_entry.copy())

    return transformed_articles

In [11]:
# def prep_news_article(this_dict, extra_words = None, exclude_words = None):
#     '''
#     takes in a dictionary representing an article and returns a dictionary that 
#     looks like this:
#             {
#              'title': 'the original title',
#              'original': original,
#              'stemmed': article_stemmed,
#              'lemmatized': article_lemmatized,
#              'clean': article_without_stopwords
#             }
#     Note that if the orignal dictionary has a title property, it will remain unchanged 
#     (same goes for the category property).
#     '''
#     # put the content section into article and make a copy
# #     article = articles['content'][article_index]    # needed for 
#     article = this_dict['content']
#     original = article

#     '''
#     apply some basic text cleaning to the string, article:
#         - lowercase everything
#         - normalize unicode characters
#         - replace anything that is not a letter, number, whitespace or a single quote
#     '''
#     article = basic_clean(article)

#     '''tokenize all the words in the string, article'''
#     article = tokenize(article)

#     '''applying stemming to all the words in the string, article'''
#     article_stemmed = stem(article)
    
#     ''''apply lemmatization to each word in the string, article'''
#     article_lemmatized = lemmatize(article)
    
#     '''remove all the stopwords, including all the words in extra_words and excluding
#     all the words in exclude list'''
#     article_without_stopwords = remove_stopwords(article, extra_words, exclude_words)

#     keys = list(this_dict.keys())
    
#     new_dict = {
#          'title': this_dict['title'],
#          'original': original,
#          'category': [this_dict['category'] if 'category' in keys else 'blog'],
#          'stemmed': article_stemmed,
#          'lemmatized': article_lemmatized,
#          'clean': article_without_stopwords
#         }
#     return new_dict

In [12]:
def prep_repo_html(this_dict, extra_words = None, exclude_words = None):
    '''
    takes in a dictionary representing an article and returns a dictionary that 
    looks like this:
            {
             'title': 'the original title',
             'original': original,
             'clean': article_without_stopwords
            }
    Note that if the orignal dictionary has a title property, it will remain unchanged 
    (same goes for the category property).
    '''
    # put the content section into article and make a copy
    article = this_dict['content']
    original = article

    '''
    apply some basic text cleaning to the string, article:
        - lowercase everything
        - normalize unicode characters
        - replace anything that is not a letter, number, whitespace or a single quote
    '''
    article = basic_clean(article)

    '''tokenize all the words in the string, article'''
    article = tokenize(article)
    
    ''''apply lemmatization to each word in the string, article'''
    article = lemmatize(article)
    
    '''remove all the stopwords, including all the words in extra_words and excluding
    all the words in exclude list'''
    article = remove_stopwords(article, extra_words, exclude_words)

    keys = list(this_dict.keys())
    
    new_dict = {
         'title': this_dict['title'],
         'category': this_dict['category'] if 'category' in keys else 'repo_readme',        
#          'original': original,
         'clean': article
        }
    return new_dict

In [13]:
def prepare_repo_html_data(articles, extra_words = None, exclude_words = None):
    # takes in the list of articles dictionaries, 
    # applies the prep_article function to each one, 
    # and returns the transformed data.
    transformed_articles = []
    for article_index in range(len(articles)):
        transformed_entry = prep_repo_html(articles[article_index], extra_words, exclude_words)
        transformed_articles.append(transformed_entry.copy())
        df = pd.DataFrame.from_dict(transformed_articles)

    return transformed_articles, df

In [14]:
# if __name__ == "__main__":
'''create a list of extra words and another of words to exclude from the stoplist'''
extra_words = ['codeup']
exclude_words = ['']

articles = get_all_readme_files_and_languages(url_list)
print()
transformed_data, df = prepare_repo_html_data(articles, extra_words, exclude_words)


In [17]:
df

,category,clean,title
0,repo_readme,welcome repository contains source code window...,microsoft/Terminal
1,repo_readme,python 100 python python c c python python jav...,jackfrued/Python-100-Days
2,repo_readme,css chat truly monstrous async web chat using ...,kkuchta/css-only-chat
3,repo_readme,overview powertoys set utilities power users t...,microsoft/PowerToys
4,repo_readme,beautifully simple click copy css effects http...,jolaleye/cssfx
5,repo_readme,english version readme click leetcode 3 4 0 1 ...,MisterBooo/LeetCodeAnimation
6,repo_readme,welcome code repository flutter web repository...,flutter/flutter_web
7,repo_readme,algorithms python algorithms implemented pytho...,TheAlgorithms/Python
8,repo_readme,make slides markdown easily write markdown cre...,hiroppy/fusuma
9,repo_readme,java offer leetcode linux http socket sql leet...,CyC2018/CS-Notes


In [15]:
transformed_data

[{'title': 'microsoft/Terminal',
  'category': 'repo_readme',
  'clean': 'welcome repository contains source code windows terminal windows console host conhost exe components shared two projects colortool sample projects show consume windows console apis build status project build status terminal colortool terminal console overview please take minutes review overview diving code windows terminal windows terminal new modern feature rich productive terminal application command line users includes many features frequently requested windows command line community including support tabs rich text globalization configurability theming styling terminal also need meet goals measures ensure remains fast efficient consume vast amounts memory power windows console host windows console host conhost exe windows original command line user experience implements windows command line infrastructure responsible hosting windows console api input engine rendering engine user preferences console host code 

In [16]:
# for index in range(len(transformed_data)):
#     print ('index:', index)
#     for key in transformed_data[index]:
#         print('key:', key)
#         print(transformed_data[index][key])
#         print('*******************')
#     print ('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
